Import <b>pandas</b> e <b>numpy</b>

In [88]:
import pandas as pd
import numpy as np

Caricamento csv <b>Users</b> in dataframe pandas

In [89]:
users = pd.read_csv(r'C:\Users\lucac\Desktop\FOCS\Users.csv', encoding = 'latin-1', sep=';')
users.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


Caricamento csv <b>Books</b> in dataframe pandas

In [90]:
books = pd.read_csv(
    r'C:\Users\lucac\Desktop\FOCS\Books.csv', 
    encoding = 'latin-1',
    sep=';',
    low_memory = False) #altrimenti errore perchè Year-Of-Publication sembra avere tipo 'misto'

books['ISBN'] = books['ISBN'].str.upper() #alcuni ISBN hanno l'ultimo carattere 'X' minuscolo
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


Caricamento csv <b>Book Ratings</b> in dataframe pandas

In [91]:
book_ratings = pd.read_csv(r'C:\Users\lucac\Desktop\FOCS\Book-Ratings.csv', encoding = 'latin-1', sep=';')

book_ratings['ISBN'] = book_ratings['ISBN'].str.upper() #alcuni ISBN hanno l'ultimo carattere 'X' minuscolo
book_ratings.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


Caricamento csv <b>Goodbooks</b> in dataframe pandas

In [92]:
goodbooks = pd.read_csv(r'C:\Users\lucac\Desktop\FOCS\goodbooks.csv', encoding = 'latin-1', sep=',')

goodbooks['isbn'] = goodbooks['isbn'].str.upper() #alcuni ISBN hanno l'ultimo carattere 'X' minuscolo
goodbooks['original_publication_year'] = goodbooks['original_publication_year'].astype('int') #conversione anno in intero
goodbooks.head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


Caricamento csv <b>Goodbooks ratings</b> in dataframe pandas

In [93]:
goodbooks_ratings = pd.read_csv(r'C:\Users\lucac\Desktop\FOCS\goodbooks-ratings.csv', encoding = 'latin-1', sep=',')
goodbooks_ratings.head(3)

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5


### 1. Normalize the location field of Users dataset, splitting into city, region, country.

Il seguente codice permette di normalizzate il campo Location in City, Region e Country. Da notare che alcuni record, nel campo Location, presentano più di tre stringhe separate da virgole: un'esplorazione dei dati ha permesso di comprendere che l'ultima stringa, solitamente, corrisponde al paese, la penultima alla regione e la terzultima alla città. Eventuali altre stringhe si riferiscono ad indirizzo e numero civico e, di conseguenza, sono state eliminate. 

In [19]:
users[['City','Region', 'Country']] = users['Location'].str.rsplit( #Reverse splitting della colonna Location in 3 nuove colonne
    ",", 
    n=2, 
    expand=True
)
users['City'] = users['City'].apply( #Salvataggio in City della substring a partire dall'ultima occorrenza di ',' (se presente)
    lambda x: x[x.rfind(',')+1:] if x.rfind(',') != -1 else x
)
users.drop( #eliminazione colonna Location
    'Location', 
    axis=1, 
    inplace= True
)
users.head(5)

,User-ID,Age,City,Region,Country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom


### 2. For each book in the Books dataset, compute its average rating.

Il seguente codice permette la computazione della media dei voti per ogni libro del dataset Books individuato dall'ISBN. 
I risultati sono salvati in un nuovo dataframe insieme ad altri campi come Size, che saranno utilizzati per richieste successive.

In [20]:
books_wratings = books.merge( #merging
    book_ratings, 
    how='left', #come left outer join
    on='ISBN'
).groupby( #raggruppamento per ISBN
    'ISBN', 
    as_index=False
).agg( #aggregation (Book-rating: media), le altre colonne sono utili per richieste successive
    {'Book-Title': 'first','Book-Author' : 'first', 'Publisher':'first', 'Book-Rating': 'mean', 'Year-Of-Publication': 'size'}
).rename( #rename colonna 'Year-Of-Publication'
    columns={'Book-Rating': 'Avg-Book-Rating', 'Year-Of-Publication':'Size'}
)

books_wratings.head(5)

,ISBN,Book-Title,Book-Author,Publisher,Avg-Book-Rating,Size
0,0000913154,The Way Things Work: An Illustrated Encycloped...,C. van Amerongen (translator),Simon &amp; Schuster,8.0,1
1,0001010565,Mog's Christmas,Judith Kerr,Collins,0.0,2
2,0001046438,Liar,Stephen Fry,Harpercollins Uk,9.0,1
3,0001046713,Twopence to Cross the Mersey,Helen Forrester,HarperCollins Publishers,0.0,1
4,000104687X,"T.S. Eliot Reading \The Wasteland\"" and Other ...",T.S. Eliot,HarperCollins Publishers,6.0,1


## 3. For each book in the GoodBooks dataset, compute its average rating.

Il codice seguente permette di computare, per ogni libro del dataset GoodBooks identificato da book_id, la media dei voti. Il campo average_rating è già presente nel dataset GoodBooks; il campo avg_ratings_gbr, invece, è calcolato grazie al dataset Goodbooks Ratings.

In [77]:
goodbooks_wratings = goodbooks[['book_id','isbn','average_rating']].merge( #merging dopo aver selezionato le 3 colonne rilevanti
    goodbooks_ratings, 
    how='left',  #come left outer join
    on='book_id'
).groupby( #raggruppamento per book_id
    'book_id', 
    as_index= False
).agg(  #aggregation (rating: media),
    {'book_id': 'first','isbn' : 'first', 'average_rating':'first', 'rating': 'mean'}
).rename( #rename colonna 'rating'
    columns={'rating': 'avg_ratings_gbr'}
)

goodbooks_wratings.head(5)

,book_id,isbn,average_rating,avg_ratings_gbr
0,1,439023483,4.34,NaN
1,2,439554934,4.44,5.0
2,3,316015849,3.57,NaN
3,4,61120081,4.25,NaN
4,5,743273567,3.89,4.0


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset merged books. The books that have not been merged together will not appear in merged books.

Il codice seguente permette di individuare tutti i libri identificati da title, author e publisher che risultano duplicati all'interno del dataset Books.

In [22]:
key_column_list = ['Book-Title','Book-Author', 'Publisher'] #chiave per l'identificazione di un libro

merged_books = books[books.duplicated( #selezione dei record duplicati sulla base della chiave
    key_column_list,
    keep=False #marking di tutti i duplicati come True
)][key_column_list].drop_duplicates( #selezione delle sole colonne di interesse / dropping duplicati collassando i record
).reset_index( #reset degli indici 
    drop=True
)

merged_books.head(5)

,Book-Title,Book-Author,Publisher
0,A Second Chicken Soup for the Woman's Soul (Ch...,Jack Canfield,Health Communications
1,OUT OF THE SILENT PLANET,C.S. Lewis,Scribner
2,Pride and Prejudice,Jane Austen,Bantam
3,All the King's Men,Robert Penn Warren,Harvest Books
4,The Dragons of Eden: Speculations on the Evolu...,Carl Sagan,Ballantine Books


### 5. For each book in merged books compute its average rating.

Il codice seguente permette di computare la media dei voti Avg-Book-Rating dei libri presenti in merged_books, sfruttando il dataframe books_wratings (con i relativi campi) e l'associatività della media.

In [23]:
key_column_list = ['Book-Title','Book-Author', 'Publisher'] #chiave per l'identificazione di un libro

merged_books_wratings = merged_books.merge( #merging merged_books con books_wratings
    books_wratings, 
    how='left', #come left outer join
    on=key_column_list #su chiave
).groupby( #raggruppamento per chiave
    key_column_list, 
    as_index=False
).apply( #calcolo media ponderata sfruttando l'associatività della media
    lambda x: pd.Series({'Avg-Book-Rating': np.average(x['Avg-Book-Rating'], weights= x['Size'])})
)

merged_books_wratings.head(5)

,Book-Title,Book-Author,Publisher,Avg-Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,3.000000
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,1.888889
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,7.000000
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3.833333
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.000000


### 6. For each book in merged books compute the minimum and maximum of the average ratings over all corresponding books in the books dataset.

Il codice seguente permette di arricchire il dataframe merged_books_wratings con i valori medi minimi e massimi delle edizioni dei libri del dataset merge_book.

In [75]:
books_wratings_grouped = books_wratings.groupby(key_column_list, as_index=False)[['Avg-Book-Rating']] #dataframe raggruppato per chiave

merged_books_wratings.merge( #merging merged_books_wratings con i minimi di books_wratings_grouped
    books_wratings_grouped.min().rename(
        columns={'Avg-Book-Rating': 'Min-Avg-Book-Rating'}
    ), 
    how='left', #come left outer join
    on=key_column_list #sulla chiave
).merge(  #merging merged_books_wratings con i massimi di books_wratings_grouped
    books_wratings_grouped.max().rename( 
        columns={'Avg-Book-Rating': 'Max-Avg-Book-Rating'}
    ), 
    how='left', #come left outer join
    on=key_column_list #sulla chiave
).head(5)

,Book-Title,Book-Author,Publisher,Avg-Book-Rating,Min-Avg-Book-Rating,Max-Avg-Book-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,3.000000,0.000000,6.00
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,1.888889,0.000000,3.40
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,7.000000,6.666667,7.25
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,3.833333,0.000000,5.00
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,0.000000,0.000000,0.00


### 7. For each book in goodbooks, compute the list of its authors. Assuming that the number of reviews with a text (column work_text_reviews_count) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the shared number of reviews with a text.

Il codice seguente permette il calcolo dell'elenco degli autori per ogni libro del dataset GoodBooks.

In [25]:
goodbooks[['book_id','isbn','authors']].head(5)

,book_id,isbn,authors
0,1,439023483,Suzanne Collins
1,2,439554934,"J.K. Rowling, Mary GrandPrÃ©"
2,3,316015849,Stephenie Meyer
3,4,61120081,Harper Lee
4,5,743273567,F. Scott Fitzgerald


Il codice seguente permette di computare, per ogni autore, il numero totale delle recensioni con un testo. Il dataset goodbooks_splitted, che sarà utile anche per richieste successive, contiene i libri del dataset GoodBooks e in ogni riga compare un singolo autore. 

In [26]:
goodbooks_splitted = goodbooks.copy() #creazione copia dataframe goodbooks

goodbooks_splitted['authors'] = goodbooks_splitted['authors'].str.split(",") #splitting colonna authors (ritorna una lista)

goodbooks_splitted = goodbooks_splitted.explode( #esplosione liste della colonna authors su più righe
    'authors'
).reset_index( #reset indici
    drop=True
).rename( #renaming colonna authors
    columns={'authors': 'author'}
)

goodbooks_splitted.groupby( #raggruppamento per autore
    'author', 
    as_index= False
)[['work_text_reviews_count']].sum().head(5) #selezione colonna work_text_reviews_count e somma

,author,work_text_reviews_count
0,Alan R. Clarke,55781
1,B.M. Mooyaart-Doubleday,20825
2,Bernard Knox,8101
3,CelÃ¢l Ãster,45518
4,David J. Skal,17263


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

Il codice seguente permette di calcolare, per ogni anno, l'autore (o gli autori) con il più alto numero di recensioni con un testo per un singolo libro (max)

In [33]:
goodbooks.loc[ #selezione delle righe in base agli indici
    goodbooks.groupby( #raggruppamento per anno 
        'original_publication_year'
    )['work_text_reviews_count'].idxmax() #selezione degli indici delle righe con valore massimo per work_text_reviews_count
][[
    'original_publication_year','authors','work_text_reviews_count'
]].sort_values( #sorting desc righe per anno 
    'original_publication_year',
    ascending = False
).reset_index( #reset indici
    drop = True
).head()

,original_publication_year,authors,work_text_reviews_count
0,2015,Paula Hawkins,93600
1,2012,John Green,140739
2,2011,Veronica Roth,101023
3,2010,Suzanne Collins,96274
4,2009,Suzanne Collins,88538


Il codice seguente permette di calcolare, per ogni anno, l'autore (o gli autori) con il più alto numero di recensioni con un testo in totale (sum)

In [34]:
goodbooks_splitted.groupby( #raggruppamento per anno e autore
    ['original_publication_year','author'], 
    as_index= False
)[['work_text_reviews_count']].sum( #selezione colonna work_text_reviews_count e somma
).groupby( #raggruppamento per anno e reviews
    ['original_publication_year','work_text_reviews_count'], 
    as_index = False
).agg( #aggregation per collassare più righe in un'unica riga, concatenando gli autori in una stringa
    {'author': lambda x: ", ".join(x)}
).rename( #renaming colonna author
    columns={'author': 'authors'}
).sort_values( #sorting decrescente delle righe per numero recensioni
    'work_text_reviews_count', 
    ascending= False
).drop_duplicates( # eliminazione delle righe duplicate per anno (rimane solo la prima, quella col numero recensioni più alte in quanto ordinate )
    'original_publication_year'
).sort_values( #sorting righe per anno 
    'original_publication_year',
    ascending = False
).reset_index( #reset indici
    drop = True
).head(10)

,original_publication_year,work_text_reviews_count,authors
0,2015,93600,Paula Hawkins
1,2012,140739,John Green
2,2011,103489,E.L. James
3,2010,96274,Suzanne Collins
4,2009,88538,Suzanne Collins
5,2008,155254,Suzanne Collins
6,2007,51942,"Mary GrandPrÃ©, J.K. Rowling"
7,2006,55732,Sara Gruen
8,2005,95009,Stephenie Meyer
9,2004,30719,Jodi Picoult


### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

Il codice seguente permette, per ogni libro identificato dal codice ISBN presente sia in Books che in GoodBooks, il calcolo della differenza Avg-Difference (in valore assoluto) tra average_rating (media relativa al dataset GoodBooks) e Avg-Book-Rating (media relativa al dataset Books).

In [84]:
books_goobooks= books_wratings.merge( # merging books_wratings con goodbooks
    goodbooks_wratings, 
    how='inner', #inner per individuare intersezione
    left_on='ISBN', 
    right_on='isbn'
)[['ISBN','average_rating','avg_ratings_gbr','Avg-Book-Rating']] #selezione delle colonne di interesse

books_goobooks['Avg-Difference']= abs(books_goobooks['average_rating'] - books_goobooks['Avg-Book-Rating']) # calcolo differenza
books_goobooks['Avg-Difference-gbr']= abs(books_goobooks['avg_ratings_gbr'] - books_goobooks['Avg-Book-Rating']) # calcolo differenza

books_goobooks.head(5)

,ISBN,average_rating,avg_ratings_gbr,Avg-Book-Rating,Avg-Difference,Avg-Difference-gbr
0,014028009X,3.75,NaN,3.751852,0.001852,NaN
1,014038572X,4.06,NaN,4.460674,0.400674,NaN
2,043965548X,4.53,5.0,3.533333,0.996667,1.466667
3,1400032717,3.85,NaN,4.813187,0.963187,NaN
4,1594480001,4.26,NaN,2.400000,1.860000,NaN


### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

Il codice seguente permette di splittare il dataframe users in più dataframe, sulla base del campo Age. Per fare questo, è stato creato un dict age_dict che contiene i dataframe.

In [72]:
age_dict = {} 

age_dict[0] = users[users['Age'].isna()] # dataframe con valori sconosciuti
age_dict[1] = users[users['Age'].between(0, 14)] #dataframe con valori da 0 a 14

for i,j in enumerate(range(15,int(users['Age'].max()),10)):
    age_dict[i+2] = users[users['Age'].between(j, j+9)] #dataframe relativi a tutti gli intervalli (step di 10 anni), fino all'età massima

### 11. Find the books that appear only in the goodbooks datasets.

Il codice seguente permette di individuare i libri che appaiono solo nel dataset GoodBooks (e non nel dataset Books)

In [73]:
goodbooks.merge( #merging goodbooks con books
    books,
    how='left', #come left outer join
    left_on='isbn',
    right_on='ISBN', 
    indicator=True #aggiunge colonna '_merge' per indicare informazioni in merito al merging
).query( #selezione delle righe 'left_only' (presenti solo in goodbooks)
    '_merge == "left_only"'
)[list(goodbooks.columns.values)].head(5) #restrizione alle sole colonne di goodbooks

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

Il codice seguente, per ogni libro del dataset Books identificato da Titolo e Autore, permette di calcolare il numero di volte che appare nel dataset, ordinando i risultati.

In [42]:
books.value_counts(subset=['Book-Title', 'Book-Author'])

Book-Title                        Book-Author            
Little Women                      Louisa May Alcott          21
Wuthering Heights                 Emily Bronte               20
Adventures of Huckleberry Finn    Mark Twain                 20
Pride and Prejudice               Jane Austen                18
The Secret Garden                 Frances Hodgson Burnett    16
                                                             ..
Heritage of Hastur                Marion Zimmer Bradley       1
Heritage of Music (4 Volume Set)  Michael Raeburn             1
Heritage of Shame                 Meg Hutchinson              1
Heritage of Stars                 Clifford D. Simak           1
Ã?Ã?thique en toc               Didier Daeninckx            1
Length: 251183, dtype: int64

### 13. Find the author with the highest average rating according to the goodbooks datasets.

Il codice seguente permette di individuare gli autori con la più alta media complessiva di voti (relativamente al dataset GoodBooks), sfrutando il dataframe goodbooks_splitted e l'associatività della media.

In [86]:
goodbooks_splitted.groupby( #raggruppamento di goodbooks_splitted per autore
    'author'
).apply(
    lambda x: pd.Series( #calcolo media ponderata sfruttando l'associatività della media
        {'Avg-Book-Rating': np.average(x['average_rating'], weights= x['ratings_count'])}
    )
).sort_values( #sorting righe per Avg-Book-Rating
    'Avg-Book-Rating', 
    ascending = False
).head(5)

,Avg-Book-Rating
author,
Rufus Beck,4.530000
Mary GrandPrÃ©,4.486117
J.K. Rowling,4.486117
Kathryn Stockett,4.450000
George R.R. Martin,4.450000
